<a href="https://colab.research.google.com/github/rahiakela/mlops-research-and-practice/blob/main/MLOps-Specialization/course-3-machine-learning-modeling-pipelines-in-production/week-3-high-performance-modeling/01_distributed_strategies_with_tf_and_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Distributed Strategies with TF and Keras

Welcome, during this ungraded lab you are going to perform a distributed training strategy using TensorFlow and Keras, specifically the [`tf.distribute.MultiWorkerMirroredStrategy`](https://www.tensorflow.org/api_docs/python/tf/distribute/MultiWorkerMirroredStrategy). 

With the help of this strategy, a Keras model that was designed to run on single-worker can seamlessly work on multiple workers with minimal code change. In particular you will:


1. Perform training with a single worker.
2. Understand the requirements for a multi-worker setup (`tf_config` variable) and using context managers for implementing distributed strategies.
3. Use magic commands to simulate different machines.
4. Perform a multi-worker training strategy.

This notebook is based on the official [Multi-worker training with Keras](https://www.tensorflow.org/tutorials/distribute/multi_worker_with_keras) notebook, which covers some additional topics in case you want a deeper dive into this topic.

[Distributed Training with TensorFlow](https://www.tensorflow.org/guide/distributed_training) guide is also available for an overview of the distribution strategies TensorFlow supports for those interested in a deeper understanding of `tf.distribute.Strategy` APIs.

Let's get started!

##Setup

In [1]:
import os
import sys
import json
import time

Before importing TensorFlow, make a few changes to the environment.

- Disable all GPUs. This prevents errors caused by the workers all trying to use the same GPU. **For a real application each worker would be on a different machine.**


- Add the current directory to python's path so modules in this directory can be imported.

In [2]:
# Disable GPUs
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Add current directory to path
if '.' not in sys.path:
  sys.path.insert(0, '.')

The previous step is important since this notebook relies on writting files using the magic command `%%writefile` and then importing them as modules.

Now that the environment configuration is ready, import TensorFlow.


In [3]:
import tensorflow as tf

print('TensorFlow Version:', tf.__version__)

# Ignore warnings
tf.get_logger().setLevel('ERROR')

TensorFlow Version: 2.7.0


## Dataset and model definition

Next create an `mnist.py` file with a simple model and dataset setup. This python file will be used by the worker-processes in this tutorial.

The name of this file derives from the dataset you will be using which is called [mnist](https://keras.io/api/datasets/mnist/) and consists of 60,000 28x28 grayscale images of the first 10 digits.

In [4]:
%%writefile mnist.py

# import os
import tensorflow as tf
import numpy as np

def mnist_dataset(batch_size):
  # load the data
  (x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
  # Normalize pixel values for x_train and cast to float32
  x_train = x_train / np.float32(255)
  # Cast y_train to int64
  y_train = y_train.astype(np.int64)
  # Define repeated and shuffled dataset
  train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(60000).repeat().batch(batch_size)
  return train_dataset

def build_and_compile_cnn_model():
  # Define simple CNN model using Keras Sequential
  model = tf.keras.Sequential([
      tf.keras.layers.InputLayer(input_shape=(28, 28)),
      tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
      tf.keras.layers.Conv2D(32, 3, activation="relu"),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation="relu"),
      tf.keras.layers.Dense(10)                         
  ])

  # compile the model
  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
                metrics=['accuracy'])
  return model

Writing mnist.py


Check that the file was succesfully created:

In [5]:
!ls *.py

mnist.py


Import the mnist module you just created and try training the model for a small number of epochs to observe the results of a single worker to make sure everything works correctly.

In [6]:
# Import your mnist model
import mnist

# Set batch size
batch_size = 64

# Load the dataset
single_worker_dataset = mnist.mnist_dataset(batch_size)

# Load compiled CNN model
single_worker_model = mnist.build_and_compile_cnn_model()

# As training progresses, the loss should drop and the accuracy should increase.
single_worker_model.fit(single_worker_dataset, epochs=3, steps_per_epoch=70)

Epoch 1/3
70/70 [==============================] - 3s 40ms/step - loss: 2.2648 - accuracy: 0.2797
Epoch 2/3
70/70 [==============================] - 3s 40ms/step - loss: 2.2021 - accuracy: 0.4808
Epoch 3/3
70/70 [==============================] - 3s 40ms/step - loss: 2.1261 - accuracy: 0.6132


Everything is working as expected! 

Now you will see how multiple workers can be used as a distributed strategy.

## Multi-worker Configuration

Now let's enter the world of multi-worker training. In TensorFlow, the `TF_CONFIG` environment variable is required for training on multiple machines, each of which possibly has a different role. `TF_CONFIG` is a JSON string used to specify the cluster configuration on each worker that is part of the cluster.

There are two components of `TF_CONFIG`: `cluster` and `task`. 

Let's dive into how they are used:

`cluster`:
- **It is the same for all workers** and provides information about the training cluster, which is a dict consisting of different types of jobs such as `worker`.

- In multi-worker training with `MultiWorkerMirroredStrategy`, there is usually one `worker` that takes on a little more responsibility like saving checkpoint and writing summary file for TensorBoard in addition to what a regular `worker` does. 
-Such a worker is referred to as the `chief` worker, and it is customary that the `worker` with `index` 0 is appointed as the chief `worker` (in fact this is how `tf.distribute.Strategy` is implemented).

`task`:
- Provides information of the current task and is different on each worker. It specifies the `type` and `index` of that worker. 

Here is an example configuration: